In [ ]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND time >= now()-"+time_interval+";"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
upload_df.head()

Are there any zeros in Upload speed colum and do they come from NaNs in the original table?

In [ ]:
mylist=['UPLOAD']
upload_df[upload_df[mylist].eq(0).all(1)]

In [ ]:
upload_df = upload_df[upload_df.UPLOAD != 0]
upload_df.shape

In [ ]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
simple_bar_plot(xvalues=points_by_device["SK_PI"],
                yvalues=points_by_device["counts"],
                name="ping datapoints",
                title="Number of data points per device")

Device 13 doesn to have any speedtest data?  
Device 5 has the largest amount of datapoints - is it on for a longest time?

#### How many datapoints in the last 4 weeks?

In [ ]:
time_interval='4w' #5d

In [ ]:
our_weeks_ago = datetime.date.today() - datetime.timedelta(days=28)
upload_df_last4weeks = upload_df[upload_df['time']>four_weeks_ago]
points_by_device_las4weeks = upload_df_last4weeks.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
merged_results=pd.merge(points_by_device, points_by_device_las4weeks, on='SK_PI', how='outer')
merged_results['counts_y']=merged_results['counts_y'].fillna(0)
merged_results['result']=merged_results['counts_x'].sub(merged_results['counts_y'], axis=0)

In [ ]:
trace1 = go.Bar(
            x=merged_results["SK_PI"],
            y=merged_results["counts_y"],
            name='Last 4 weeks',
    )
trace2= go.Bar(
            x=merged_results["SK_PI"],
            y=merged_results['result'],
            name='The rest of the time',
    )
data=[trace1, trace2]
layout = go.Layout(
        barmode='stack',
        title="Number of data points per device: last 4 weeks vs entire time"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Devices 1,2,4,6 and 8 have data but not in the last 4 weeks.  
#### When  all the devices started/ended reporting:

In [ ]:
start_times=upload_df.groupby('SK_PI')['time'].min()
end_times=upload_df.groupby('SK_PI')['time'].max()
device_numbers=sorted(upload_df['SK_PI'].unique())
#print(device_numbers)
data=[]
for device in device_numbers:
    time_last=end_times[device]
    time_first=start_times[device]
    print("Device: ", device,"  was reporting from ", time_first, " to ", time_last)
    trace = go.Scatter(x=[time_first,time_last],y=[device,device], name = device)
    data.append(trace)
layout = dict(title = "Device reporting times")
fig = go.Figure(data=data, layout=layout)
iplot(fig)